In [1]:
from src.standarice_data import TenisModelHandler
import pandas as pd

In [5]:
# Cargar datos de entrenamiento
df_train = pd.read_csv('../data\input\scraped\df_games.csv', delimiter='|')

# Crear instancia de la clase Preprocessing
preprocessor = TenisModelHandler(df_train, models_path='../models')

# Preprocesar datos de entrenamiento
preprocessor.preprocessing_train()

# Entrenar modelos
models_to_train = ['xgboost']
preprocessor.train_models(models_to_train)

<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
C:\Users\Julián Pérez\AppData\Local\Temp\ipykernel_19296\2379427704.py:2: SyntaxWarning: invalid escape sequence '\i'
  df_train = pd.read_csv('../data\input\scraped\df_games.csv', delimiter='|')
g:\dev\tennis_bet\code\src\standarice_data.py:304: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  player_data['weight'].fillna(mean_values['weight'], inplace=True)
g:\dev\tennis_bet\code\src\standarice_data.py:305: FutureWarning: A value

Entrenando xgboost...


g:\dev\tennis_bet\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:13:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Mejores hiperparámetros xgboost: {'model__learning_rate': 0.1, 'model__max_depth': 7, 'model__n_estimators': 200}
Accuracy en entrenamiento xgboost: 0.7502
Accuracy en prueba para xgboost: 0.7512
Reporte de clasificación para xgboost:
              precision    recall  f1-score   support

           0       0.75      0.75      0.75      8420
           1       0.75      0.75      0.75      8472

    accuracy                           0.75     16892
   macro avg       0.75      0.75      0.75     16892
weighted avg       0.75      0.75      0.75     16892



In [4]:
# Cargar nuevos datos para predicción
df_new = pd.read_csv('../data/input/scraped/df_testing.csv', delimiter='|')

# Crear instancia de la clase Preprocessing
preprocessor = TenisModelHandler(df_new, models_path='../models')

# Generar predicciones con el modelo deseado
resultado = preprocessor.predict_values('xgboost')

# Mostrar el DataFrame con las predicciones
print(resultado.head())

                  Location  pl1_bet  pl2_bet          pl1_name       pl1_flag  \
1  Gran Canaria challenger     3.03     1.35       Oliveira G.      Venezuela   
3  Gran Canaria challenger     1.22     4.00          Mager G.          Italy   
4  Gran Canaria challenger     2.20     1.63          Rosol L.  CzechRepublic   
5  Gran Canaria challenger     2.21     1.63  Gimeno Valero C.          Spain   
6  Gran Canaria challenger     1.76     2.00         Nikles J.    Switzerland   

   pl1_year_pro  pl1_weight  pl1_height pl1_hand  pl1_age         pl2_name  \
1        2010.0        79.0       185.0     left     29.0       Bourgue M.   
3        2010.0        77.0       188.0    right     29.0  Toledo Bague P.   
4        2002.0        83.0       193.0    right     39.0     Giannessi A.   
5        2016.0        80.0       188.0    right     23.0        Skatov T.   
6        2013.0        68.0       170.0    right     27.0          Gaio F.   

     pl2_flag  pl2_year_pro  pl2_weight  pl2